POS Filter

In [1]:
#1. rutin1 import module
import pandas as pd
import os
import sys
import warnings
import matplotlib.pyplot as plt
import numpy as np
import re
import string

import networkx as nx
from collections import defaultdict
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity

warnings.simplefilter(action='ignore', category=UserWarning)

In [2]:
#2. rutin2 membuat syspath ke root utk aktifkan __init__.py
repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))
sys.path.append(repo_root)

#3. rutin3 Load the dataset
dataset_path = os.path.join(repo_root, "data/dataset_ekstraksi_r30_lg.xlsx")
df = pd.read_excel(dataset_path)
df["text"] = df["judul"] +". "+ df["isi"]

In [3]:
#3 Preprocess
'''
stopwords tidak masuk dalam preprocessing
'''
def preprocess(text):
    text = re.sub(r"\b[a-zA-Z]\b", "", text)
    text = text.replace('.', '. ')
    text = re.sub('[^a-zA-Z.]', ' ', text)
    text = text.lower()
    text = re.sub("(\\d|\\W)+"," ",text)
    text = text.strip()

    return text

df["text"] = df['text'].apply(preprocess)
df["judul"] = df["judul"].apply(preprocess)

In [4]:
#4 Load stopword
def load_stopwords(repo_root, stopwords_file_path):
    """
    Load stopwords from a file located in the repository.

    :param repo_root: Root directory of the repository.
    :param stopwords_file_path: Relative path to the stopwords file from the repo_root.
    :return: Set of stopwords.
    """
    stopwords_path = os.path.join(repo_root, stopwords_file_path)
    with open(stopwords_path, 'r') as file:
        stopwords = set(file.read().strip().splitlines())
    return stopwords

repo_root = repo_root
stopwords = load_stopwords(repo_root, "notebooks/stopwords_tuning/all_stop_words.txt")

In [5]:
# Fungsi Phrase Detection

from collections import Counter
from nlp_id_local.tokenizer import PhraseTokenizer 
from nlp_id_local.postag import PosTag
from nltk.util import ngrams

def generate_ngrams(words, n=2):
    """Generate ngrams from a list of words."""
    return [" ".join(gram) for gram in ngrams(words, n)]

def detect_bigram(text):
    
    tokenizer = PhraseTokenizer()
    phrases = tokenizer.tokenize(text)
    # Include only bigrams whose individual words are in available_tokens
    bigrams_only = [phrase for phrase in phrases if phrase.count(" ") == 1]

    return bigrams_only

def detect_trigram(text):

    tokenizer = PhraseTokenizer()
    phrases = tokenizer.tokenize(text)
    # Include only trigrams whose individual words are in available_tokens
    trigrams_only = [phrase for phrase in phrases if phrase.count(" ") == 2 ]

    return trigrams_only

"""
# Function to incorporate bigrams and trigrams in the correct sequence, existing phrase contructor deleted
def incorporate_bigrams_trigrams(unigrams, bigrams, trigrams):
    combined_tokens = []
    skip = 0

    for i in range(len(unigrams)):
        if skip > 0:
            skip -= 1
            continue

        bigram_formed = ' '.join(unigrams[i:i+2]) in bigrams
        trigram_formed = ' '.join(unigrams[i:i+3]) in trigrams

        if bigram_formed:
            combined_tokens.append(' '.join(unigrams[i:i+2]))
            skip = 1
        elif trigram_formed:
            combined_tokens.append(' '.join(unigrams[i:i+3]))
            skip = 2
        else:
            combined_tokens.append(unigrams[i])

    return combined_tokens

# Function to incorporate bigrams and trigrams in the correct sequence, existing phrase contructor not deleted
def incorporate_bigrams_trigrams(unigrams, bigrams, trigrams):
    combined_tokens = []

    for i in range(len(unigrams)):
        bigram_formed = ' '.join(unigrams[i:i+2]) in bigrams
        trigram_formed = ' '.join(unigrams[i:i+3]) in trigrams

        if trigram_formed:
            combined_tokens.append(' '.join(unigrams[i:i+3]))
        elif bigram_formed:
            combined_tokens.append(' '.join(unigrams[i:i+2]))

        combined_tokens.append(unigrams[i])

    return combined_tokens
"""

# Function to incorporate bigrams and trigrams in the correct sequence twice beginning and end, existing phrase constructor not deleted
def incorporate_bigrams_trigrams(unigrams, bigrams, trigrams):
    combined_tokens = []

    for i in range(len(unigrams)):
        bigram_formed = ' '.join(unigrams[i:i+2]) in bigrams
        trigram_formed = ' '.join(unigrams[i:i+3]) in trigrams

        if trigram_formed:
            phrase = ' '.join(unigrams[i:i+3])
            combined_tokens.append(phrase)  # Append before the unigrams
            combined_tokens.extend(unigrams[i:i+3])
            combined_tokens.append(phrase)  # Append after the unigrams
        elif bigram_formed:
            phrase = ' '.join(unigrams[i:i+2])
            combined_tokens.append(phrase)  # Append before the unigrams
            combined_tokens.extend(unigrams[i:i+2])
            combined_tokens.append(phrase)  # Append after the unigrams
        else:
            combined_tokens.append(unigrams[i])

    return combined_tokens


def detect_all_tokens(text):
    unigrams = [word for word in text.split()]
    bigrams = detect_bigram(text)
    trigrams = detect_trigram(text)
    
    # Incorporating bigrams and trigrams into the sequence of tokens
    all_tokens = incorporate_bigrams_trigrams(unigrams, bigrams, trigrams)

    return all_tokens

# example
text = df["text"][1]
words = detect_all_tokens(text)
print(text)
print("================================================================")
print(words)

template document jtb gpf project mengacu kepada dokumen jtb cp ctr exhibit coordination procedure kami sampaikan template document yang akan dipergunakan pada proyek jambaran tiung biru jtb gas processing facilities gpf demikian disampaikan sebagai acuan pengelolaan dokumen atas perhatiannya kami ucapkan terima kasih
['template document', 'template', 'document', 'template document', 'document', 'jtb', 'gpf', 'project', 'mengacu', 'kepada', 'dokumen', 'jtb', 'cp', 'ctr', 'exhibit', 'coordination', 'procedure', 'kami', 'sampaikan', 'template document', 'template', 'document', 'template document', 'document', 'yang', 'akan', 'dipergunakan', 'pada', 'proyek', 'jambaran', 'tiung', 'biru', 'jtb', 'gas', 'processing', 'facilities', 'gpf', 'demikian', 'disampaikan', 'sebagai', 'acuan', 'pengelolaan', 'dokumen', 'atas', 'perhatiannya', 'kami', 'ucapkan', 'terima', 'kasih']


In [6]:
# Load Word2vec Model
w2v_model_path = os.path.join(repo_root, "models/embedding_model/w2v_wiki_own_phrase_training_200.model")

def load_word2vec_model(model_path):
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"The provided Word2Vec model path does not exist: {model_path}")
    
    w2v_model = Word2Vec.load(model_path) 
    available_tokens = set(w2v_model.wv.key_to_index)
    
    return w2v_model, available_tokens
    
w2v_model, available_tokens = load_word2vec_model(w2v_model_path)

# Function to compose vector for a phrase if not defined on the w2v model
def get_phrase_embedding(phrase, w2v_model):
    """Get the averaged word embedding for a phrase."""
    words = phrase.split()
    embeddings = [w2v_model.wv[word] for word in words if word in w2v_model.wv.key_to_index]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return None
    
# Example
phrase = "bekerja sama"
phrase_vector = get_phrase_embedding(phrase, w2v_model)
phrase_vector

array([-0.10741319,  0.73422295, -0.13234389,  2.1272323 , -2.411738  ,
       -1.6032877 ,  1.2794459 , -0.55809057, -0.2812829 , -0.9950116 ,
        3.5894551 , -1.2721555 ,  0.1925945 ,  0.3263977 , -0.3885795 ,
       -0.29037327, -0.64925265,  0.5564457 ,  0.33676478, -0.07207242,
       -1.7134237 ,  1.9276747 , -0.32966518, -0.07611573,  1.154696  ,
       -2.8427987 ,  0.9857985 , -0.04882255, -0.73466855, -2.0622673 ,
        0.07548565, -1.5615342 , -2.3783605 ,  1.6252084 , -1.4792495 ,
        1.4381717 ,  1.1845043 ,  1.7329456 , -1.2029533 , -1.8848159 ,
       -3.9125085 ,  0.94502825,  1.0513016 , -0.4570459 , -0.05512244,
        2.3047898 , -3.4308467 , -0.22764379, -1.3446113 , -0.8263667 ,
       -1.0181042 ,  0.5820242 ,  1.4395669 , -0.36169428,  0.6613957 ,
       -0.08114982,  0.58890176, -0.16885197, -1.0263593 ,  1.3066081 ,
       -0.01349652, -2.5810575 ,  2.7768462 , -0.01583394, -0.14296976,
       -3.2206645 , -2.5853512 ,  0.18093038,  0.15952486, -0.24

In [7]:
def get_cosine_similarity(w1, w2, w2v_model):
    vec1 = get_phrase_embedding(w1, w2v_model)
    vec2 = get_phrase_embedding(w2, w2v_model)

    if vec1 is None or vec2 is None:
        return 0

    similarity = cosine_similarity([vec1], [vec2])[0][0]
    return similarity

# example, seharusnya dekat hubungan 2 vector ini
w1 = 'gotong royong'
w2 = 'bekerja sama'
w2v_model, available_tokens = load_word2vec_model(w2v_model_path)
cos_sim = get_cosine_similarity(w1, w2, w2v_model)
cos_sim

0.010565544

In [8]:
# Utilities
'''
def get_unique_tokens_pos(all_tokens, pos_model_path):
    """
    Get unique POS tags for tokens.
    """
    pos_model_path = os.path.join(repo_root, "notebooks/nlp-id_retraining/train_tuned.pkl")
    postagger = PosTag(pos_model_path)
    pos_tokens = []
    seen_tokens = set()
    
    for token in all_tokens:
        if token not in seen_tokens:
            seen_tokens.add(token)
            tokens_pos = postagger.get_phrase_tag(token)
            pos_tokens.append(tokens_pos)
    return pos_tokens
'''
# Fungsi Visualisasi
def visualize_graph(G, labels):
    # Remove self-loops (edges that connect a node to itself)
    G.remove_edges_from(nx.selfloop_edges(G))

    fig = plt.figure(figsize=(12, 12))
    pos = nx.spring_layout(G, seed=42)
    #nx.draw(G, pos=pos, with_labels=False, font_weight="bold", node_size=5000, node_color='skyblue')
    nx.draw(G, pos=pos, with_labels=False, font_weight="bold")
    nx.draw_networkx_labels(G, pos, labels, font_size=12)
    plt.show()

TEXTRANK and ADJUSTMENT

In [9]:
# Postagging Function
def postag_tokens(tokens):
    pos_model_path = os.path.join(repo_root, "notebooks/nlp-id_retraining/train_tuned.pkl")
    postagger = PosTag(pos_model_path)
    pos_tokens = [postagger.get_phrase_tag(token) for token in tokens]
    # Flatten the list
    return [item for sublist in pos_tokens for item in sublist]

# Attach POS Tags to Keywords
def attach_pos_tags(keywords, pos_tokens):
    pos_dict = dict(pos_tokens)
    return [(word, score, pos_dict.get(word, 'UNK')) for word, score in keywords]

# Filter by Selected POS Tags
def filter_by_pos(keywords_with_pos, selected_pos):
    return [item for item in keywords_with_pos if item[2] in selected_pos]

#example
text = df["text"][1]
words = detect_all_tokens(text)
pos_tokens = postag_tokens(words)
#print(words)
print(pos_tokens)

[('template document', 'NP'), ('template', 'FW'), ('document', 'NNP'), ('template document', 'NP'), ('document', 'NNP'), ('jtb', 'SC'), ('gpf', 'NN'), ('project', 'NN'), ('mengacu', 'VP'), ('kepada', 'IN'), ('dokumen', 'NN'), ('jtb', 'SC'), ('cp', 'NN'), ('ctr', 'NN'), ('exhibit', 'NN'), ('coordination', 'FW'), ('procedure', 'NN'), ('kami', 'PR'), ('sampaikan', 'VP'), ('template document', 'NP'), ('template', 'FW'), ('document', 'NNP'), ('template document', 'NP'), ('document', 'NNP'), ('yang', 'PR'), ('akan', 'ADV'), ('dipergunakan', 'VP'), ('pada', 'IN'), ('proyek', 'NN'), ('jambaran', 'NN'), ('tiung', 'NN'), ('biru', 'ADJP'), ('jtb', 'SC'), ('gas', 'NN'), ('processing', 'NN'), ('facilities', 'NN'), ('gpf', 'NN'), ('demikian', 'PR'), ('disampaikan', 'VP'), ('sebagai', 'IN'), ('acuan', 'NN'), ('pengelolaan', 'NN'), ('dokumen', 'NN'), ('atas', 'IN'), ('perhatian', 'NN'), ('nya', 'PR'), ('kami', 'PR'), ('ucapkan', 'VP'), ('terima', 'VP'), ('kasih', 'NN')]


In [10]:
# Build Co-occurrence Matrix
def build_co_occurrence_matrix(words, window_size=3):
    co_occurrence = defaultdict(int)
    for i in range(len(words) - window_size + 1):
        window = words[i:i+window_size]
        for j in range(window_size):
            for k in range(j+1, window_size):
                w1, w2 = sorted([window[j], window[k]])
                if w1 != w2:
                    co_occurrence[(w1, w2)] += 1
    return co_occurrence

# Build Graph and Compute TextRank
def build_graph_and_compute_textrank(co_occurrence, w2v_model):
    G = nx.Graph()
    for (w1, w2), weight1 in co_occurrence.items():
        weight2 = get_cosine_similarity(w1, w2, w2v_model)
        weight3 = weight1 * weight2
        if weight2 > 0:
            G.add_edge(w1, w2, weight=weight3)
    return nx.pagerank(G)

# Example
co_occurrence = build_co_occurrence_matrix(words)
#co_occurrence

scores = build_graph_and_compute_textrank(co_occurrence, w2v_model)
print("mengandung stopword:", scores)

mengandung stopword: {'template': 0.030885930577214817, 'template document': 0.05883718185645558, 'document': 0.055299738725117675, 'jtb': 0.03736840709876451, 'gpf': 0.024522394845921804, 'project': 0.006396406614423802, 'kepada': 0.016472771517533003, 'mengacu': 0.00887723137533912, 'dokumen': 0.04411615582953661, 'cp': 0.020745602274276596, 'ctr': 0.030370871004512093, 'exhibit': 0.030385369119198945, 'coordination': 0.03161413565037003, 'procedure': 0.021506565218555946, 'kami': 0.037502876595947775, 'sampaikan': 0.013107286090385084, 'yang': 0.02754576420421596, 'akan': 0.03582000706460247, 'dipergunakan': 0.012914063329421258, 'pada': 0.03087709304594505, 'proyek': 0.018422090821895186, 'jambaran': 0.02714542913894811, 'tiung': 0.03173716553968021, 'biru': 0.020632368864854367, 'gas': 0.01607436110205208, 'processing': 0.029267714520183744, 'facilities': 0.02691973787343473, 'demikian': 0.016572075181813247, 'disampaikan': 0.022411935741149, 'sebagai': 0.028810349182739066, 'acua

In [11]:
def filter_and_rank_keywords(scores, stopwords, num_keywords=10, bigram_weight=1, trigram_weight=1):
    # Adjust scores for bigrams and trigrams
    adjusted_scores = {}
    for token, score in scores.items():
        token_count = len(token.split())
        if token_count == 2:  # Bigram
            adjusted_score = score * bigram_weight
        elif token_count == 3:  # Trigram
            adjusted_score = score * trigram_weight
        else:  # Unigram
            adjusted_score = score
        adjusted_scores[token] = adjusted_score

    # Sort and filter words based on adjusted scores
    ranked_words = sorted(((score, word) for word, score in adjusted_scores.items()), reverse=True)
    ranked_words_nstopword = [(score, word) for score, word in ranked_words if word not in stopwords]
    
    keyphrases_with_scores = []
    seen_tokens = set()
    for score, token in ranked_words_nstopword:
        if token not in seen_tokens:
            keyphrases_with_scores.append((token, score))
            seen_tokens.add(token)
            if len(keyphrases_with_scores) >= num_keywords:
                break

    return keyphrases_with_scores

#Example
num_keywords=10
bigram_weight=2 
trigram_weight=3
#keyphrases_with_scores = filter_and_rank_keywords(scores, stopwords)
keyphrases_with_scores = filter_and_rank_keywords(scores, stopwords, num_keywords, bigram_weight, trigram_weight)

print("filter stopword :",keyphrases_with_scores)


filter stopword : [('template document', 0.11767436371291116), ('document', 0.055299738725117675), ('acuan', 0.03717564545266919), ('tiung', 0.03173716553968021), ('coordination', 0.03161413565037003), ('template', 0.030885930577214817), ('exhibit', 0.030385369119198945), ('ctr', 0.030370871004512093), ('processing', 0.029267714520183744), ('pengelolaan', 0.028238776959943777)]


#SAMPLE SINGLE DATASET

bikin teknik Vector composition


ALL DATASETS

In [12]:
# Load Word2Vec Model
w2v_model_path = os.path.join(repo_root, "models/embedding_model/w2v_wiki_own_phrase_training_200.model")
w2v_model, available_tokens = load_word2vec_model(w2v_model_path)

predict_textrank = pd.DataFrame()
for i in df.index:
#for i in df.loc[2:5].index:    
    print('Processing index', i, end='...! ')
    # Processing Text
    text = df["text"][i]
    words = detect_all_tokens(text)
    # Postagging
    pos_tokens = postag_tokens(words)
    # Building Co-occurrence Matrix
    co_occurrence = build_co_occurrence_matrix(words)
    # Building Graph and Computing TextRank
    scores = build_graph_and_compute_textrank(co_occurrence, w2v_model)
    # Filtering and Ranking Keywords
    keyphrases_with_scores = filter_and_rank_keywords(scores, stopwords)
    # Attaching POS Tags
    keywords_with_pos = attach_pos_tags(keyphrases_with_scores, pos_tokens)
    # Filtering by Selected POS Tags
    selected_pos_tags = {'NN', 'NNP', 'VB', 'NP', 'VP', 'FW'}
    filtered_list = filter_by_pos(keywords_with_pos, selected_pos_tags)

    df_keyphrases = pd.DataFrame(filtered_list, columns=['Keyword', 'Score', 'pos'])
    a = pd.DataFrame(df_keyphrases.Keyword).T.reset_index(drop=True)
    a = a.reindex(columns=range(10), fill_value=0)
    b = pd.DataFrame(df_keyphrases.Score).round(3).T.reset_index(drop=True)
    b = b.reindex(columns=range(10), fill_value=0)
    c = pd.DataFrame(df_keyphrases.pos).round(3).T.reset_index(drop=True)
    c = c.reindex(columns=range(10), fill_value=0)
    df_keyphrases = pd.concat([a, b, c], axis=1)

    # Check if there are missing columns and add them with zero values
    missing_columns = 30 - df_keyphrases.shape[1]
    for _ in range(missing_columns):
        df_keyphrases[df_keyphrases.shape[1]] = 0

    df_keyphrases.columns = ['key_1', 'key_2','key_3', 'key_4', 'key_5','key_6', 'key_7', 'key_8','key_9','key_10',
                             'score_1', 'score_2','score_3','score_4', 'score_5','score_6','score_7', 'score_8','score_9','score_10',
                             'pos_1', 'pos_2','pos_3', 'pos_4', 'pos_5','pos_6', 'pos_7', 'pos_8','pos_9','pos_10'] 
    predict_textrank = pd.concat([predict_textrank, df_keyphrases], ignore_index=True)
    print('Done')
predict_textrank.head(3)

Processing index 0...! 

Done
Processing index 1...! Done
Processing index 2...! Done
Processing index 3...! Done
Processing index 4...! Done
Processing index 5...! Done
Processing index 6...! Done
Processing index 7...! Done
Processing index 8...! Done
Processing index 9...! Done
Processing index 10...! Done
Processing index 11...! Done
Processing index 12...! Done
Processing index 13...! Done
Processing index 14...! Done
Processing index 15...! Done
Processing index 16...! Done
Processing index 17...! Done
Processing index 18...! Done
Processing index 19...! Done
Processing index 20...! Done
Processing index 21...! Done
Processing index 22...! Done
Processing index 23...! Done
Processing index 24...! Done
Processing index 25...! Done
Processing index 26...! Done
Processing index 27...! Done
Processing index 28...! Done
Processing index 29...! Done
Processing index 30...! Done
Processing index 31...! Done
Processing index 32...! Done
Processing index 33...! Done
Processing index 34...! Done
Processing index 3

,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,...,pos_1,pos_2,pos_3,pos_4,pos_5,pos_6,pos_7,pos_8,pos_9,pos_10
0,personil penting,personil,email tersendiri,organisasi,fase,usulan personil penting,penilaian teknis,tanggal efektif,tender,0,...,NP,NN,NP,NN,NN,NP,NP,NP,NN,0
1,template document,document,acuan,tiung,coordination,template,exhibit,ctr,processing,pengelolaan,...,NP,NNP,NN,NN,FW,FW,NN,NN,NN,NN
2,kantor,ruangan,usulan,change inquiry,tidak mengalami,scope,inquiry,artikel,lokasi,0,...,NN,NN,NN,NP,VP,FW,FW,NN,NN,0


EVALUATION

In [13]:
from utils import eval

targets = df[["k1", "k2", "k3","k4", "k5", "k6","k7"]].values.tolist()
df_targets = pd.DataFrame(targets)

In [14]:
# Evaluation TextRank top 10
predict_textrank_list_10 = predict_textrank[['key_1','key_2','key_3', 'key_4','key_5','key_6', 'key_7','key_8','key_9', 'key_10']].values.tolist()
eval_textrank_10 = eval(predict_textrank_list_10, targets, True).round(3)
eval_textrank_10.columns = ['key_1','key_2','key_3', 'key_4','key_5','key_6', 'key_7','key_8','key_9', 'key_10','strict_recall', 'strict_prec', 'flex_recall','flex_prec']
eval_textrank_10 = eval_textrank_10[['key_1','key_2','key_3', 'key_4','key_5','key_6', 'key_7','key_8','key_9', 'key_10', 'flex_recall','flex_prec']] # untuk menyederhanakan hasil evaluasi
#eval_textrank_10.head(3)

# Calculate TextRank Score, using flexible score : exact maatch =1, partial match = 1, no match = 0
textrank_recall_10 = eval_textrank_10['flex_recall'].mean()
textrank_prec_10 = eval_textrank_10['flex_prec'].mean()
textrank_f1_10 = 2 * (textrank_prec_10 * textrank_recall_10) / (textrank_prec_10 + textrank_recall_10)

# Create a DataFrame with the scores
summary_10 = pd.DataFrame({'textrank': [textrank_recall_10, textrank_prec_10, textrank_f1_10]}, index=['recall', 'precision', 'F1'])
summary_10 = summary_10.round(3)
#summary_10

In [15]:
# Evaluation TextRank top 5
predict_textrank_list_5 = predict_textrank[['key_1','key_2','key_3', 'key_4','key_5']].values.tolist()
eval_textrank_5 = eval(predict_textrank_list_5, targets, True).round(3)
eval_textrank_5.columns = ['key_1','key_2','key_3', 'key_4','key_5','strict_recall', 'strict_prec', 'flex_recall','flex_prec']
eval_textrank_5 = eval_textrank_5[['key_1','key_2','key_3', 'key_4','key_5', 'flex_recall','flex_prec']] # untuk menyederhanakan hasil evaluasi
#eval_textrank_5.head(3)

# Calculate TextRank Score, using flexible score : exact maatch =1, partial match = 1, no match = 0
textrank_recall_5 = eval_textrank_5['flex_recall'].mean()
textrank_prec_5 = eval_textrank_5['flex_prec'].mean()
textrank_f1_5 = 2 * (textrank_prec_5 * textrank_recall_5) / (textrank_prec_5 + textrank_recall_5)

# Create a DataFrame with the scores
summary_5 = pd.DataFrame({'textrank': [textrank_recall_5, textrank_prec_5, textrank_f1_5]}, index=['recall', 'precision', 'F1'])
summary_5 = summary_5.round(3)
summary_5

,textrank
recall,0.179
precision,0.251
F1,0.209


In [16]:
# Evaluation TextRank top 3
predict_textrank_list_3 = predict_textrank[['key_1','key_2','key_3']].values.tolist()
eval_textrank_3 = eval(predict_textrank_list_3, targets, True).round(3)
eval_textrank_3.columns = ['key_1', 'key_2','key_3','strict_recall', 'strict_prec', 'flex_recall','flex_prec']
eval_textrank_3 = eval_textrank_3[['key_1', 'key_2','key_3', 'flex_recall','flex_prec']] # untuk menyederhanakan hasil evaluasi
#eval_textrank_3.head(3)

# Calculate TextRank Score, using flexible score : exact maatch =1, partial match = 1, no match = 0
textrank_recall_3 = eval_textrank_3['flex_recall'].mean()
textrank_prec_3 = eval_textrank_3['flex_prec'].mean()
textrank_f1_3 = 2 * (textrank_prec_3 * textrank_recall_3) / (textrank_prec_3 + textrank_recall_3)

# Create a DataFrame with the scores
summary_3 = pd.DataFrame({'textrank': [textrank_recall_3, textrank_prec_3, textrank_f1_3]}, index=['recall', 'precision', 'F1'])
summary_3 = summary_3.round(3)
summary_3

,textrank
recall,0.115
precision,0.268
F1,0.161


In [17]:
# Combine dataframe predict_textrank, df_targets and eval_textrank
predict_textrank_10 = pd.concat([predict_textrank, df_targets, eval_textrank_10], axis=1)
#predict_textrank_10.head(3)

# Combine dataframe predict_textrank, df_targets and eval_textrank
predict_textrank_5 = pd.concat([predict_textrank, df_targets, eval_textrank_5], axis=1)
#predict_textrank_5.head(3)

# Combine dataframe predict_textrank, df_targets and eval_textrank
predict_textrank_3 = pd.concat([predict_textrank, df_targets, eval_textrank_3], axis=1)
predict_textrank_3.head(3)

,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,...,2,3,4,5,6,key_1,key_2,key_3,flex_recall,flex_prec
0,personil penting,personil,email tersendiri,organisasi,fase,usulan personil penting,penilaian teknis,tanggal efektif,tender,0,...,usulan,pengganti,NaN,NaN,NaN,no_match,no_match,no_match,0.000,0.000
1,template document,document,acuan,tiung,coordination,template,exhibit,ctr,processing,pengelolaan,...,acuan,pengelolaan,dokumen,NaN,NaN,full_match,partial_match,full_match,0.429,1.000
2,kantor,ruangan,usulan,change inquiry,tidak mengalami,scope,inquiry,artikel,lokasi,0,...,lingkup kerja,akomodasi,services for company,exhibit a,NaN,partial_match,no_match,no_match,0.143,0.333


In [18]:
import pandas as pd
from openpyxl import load_workbook

def write_excel(df, sheet_name, filename):
    """
    Writes the given dataframe to an excel file with the given filename and sheet name.
    If the sheet already exists in the file, the data in the sheet will be overwritten.
    """
    try:
        # Try to load the existing workbook
        with pd.ExcelWriter(filename, engine='openpyxl', mode='a') as writer:
            if sheet_name in writer.book.sheetnames:
                # If sheet already exists, remove it
                sheet = writer.book[sheet_name]
                writer.book.remove(sheet)

            # Write the dataframe to the excel file
            df.to_excel(writer, sheet_name=sheet_name, index=False)

    except FileNotFoundError:
        # If the file doesn't exist, create a new workbook
        with pd.ExcelWriter(filename, engine='openpyxl', mode='w') as writer:
            df.to_excel(writer, sheet_name=sheet_name, index=False)


In [19]:
# Write predictions to excel file
#from utils import write_excel

sheet_name_10 = 'SE134_adjusted_textrank_10'
sheet_name_5 = 'SE134_adjusted_textrank_5'
sheet_name_3 = 'SE134_adjusted_textrank_3'

output_file = 'SE134_adjusted_textrank.xlsx'
write_excel(predict_textrank_10, sheet_name_10, output_file)
write_excel(predict_textrank_5, sheet_name_5, output_file)
write_excel(predict_textrank_3, sheet_name_3, output_file)